<a href="https://colab.research.google.com/github/lucasfreire01/Microbiness_godaddy/blob/main/godaddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drawing datas of census DataFrame**

---



In [1]:
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184112 sha256=86579dd8a63de7caa92f92e109c9a04a4ca708e548046c14821806438461847e
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


In [2]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split,RepeatedKFold
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
import matplotlib.pyplot as plt
import xgboost

In [4]:
#1St import dataset
census =  pd.read_csv('/content/drive/MyDrive/census_starter.csv')

In [5]:
census

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,82.2,82.4,84.0,86.7,88.4,56037,15.3,15.2,14.8,13.7,...,0.6,0.6,1.0,0.9,1.0,71083,73008.0,74843,73384.0,76668.0
3138,83.5,85.9,87.1,89.1,90.5,56039,37.7,37.8,38.9,37.2,...,0.7,1.2,1.4,1.5,2.0,80049,83831.0,84678,87053.0,94498.0
3139,83.8,88.2,89.5,91.4,90.6,56041,11.9,10.5,11.1,12.6,...,1.2,1.2,1.4,1.7,0.9,54672,58235.0,63403,72458.0,75106.0
3140,76.4,78.3,78.2,82.8,85.4,56043,15.4,15.0,15.4,15.0,...,1.3,1.0,0.9,0.9,1.1,51362,53426.0,54158,57306.0,62271.0


In [6]:
census = census.dropna()

In [7]:
census_train = census.drop('cfips',axis=1)

In [8]:
census_test = census['cfips'].values

In [9]:
census_train = census_train.astype('float64')
census_test = census_test.astype('float64')

In [10]:
#Detecting outliers
from pyod.models.knn import KNN
detector = KNN()
detector.fit(census_train)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [11]:
predict = detector.labels_
len(predict)

3138

In [12]:
np.unique(predict, return_counts=True)

(array([0, 1]), array([2824,  314]))

In [13]:
trust = detector.decision_scores_
trust

array([2190.63502894, 1515.9666619 , 2388.08133865, ..., 5596.49967033,
       1403.89975782, 7347.7299406 ])

In [14]:
predict

array([0, 0, 0, ..., 1, 0, 1])

In [15]:
outliers = []
for i in range(len(predict)):
  if predict[i] == 1:
    outliers.append(i)

In [16]:
print(outliers)

[5, 31, 59, 67, 68, 71, 72, 74, 75, 76, 77, 80, 83, 85, 91, 92, 93, 131, 150, 183, 185, 186, 191, 205, 222, 225, 227, 228, 250, 252, 253, 257, 260, 261, 262, 263, 267, 268, 270, 272, 276, 283, 290, 292, 301, 302, 303, 310, 313, 351, 381, 384, 389, 415, 417, 431, 443, 451, 471, 478, 480, 483, 493, 499, 503, 513, 518, 519, 525, 527, 528, 537, 538, 547, 556, 557, 566, 582, 585, 590, 592, 600, 615, 626, 775, 813, 855, 903, 933, 934, 937, 951, 970, 972, 984, 986, 1003, 1056, 1065, 1071, 1082, 1086, 1092, 1099, 1100, 1110, 1115, 1118, 1123, 1129, 1141, 1149, 1152, 1157, 1195, 1204, 1206, 1208, 1209, 1225, 1226, 1425, 1427, 1431, 1434, 1459, 1471, 1477, 1567, 1594, 1602, 1607, 1609, 1613, 1631, 1636, 1638, 1655, 1656, 1657, 1669, 1677, 1687, 1690, 1705, 1709, 1710, 1711, 1717, 1720, 1722, 1724, 1738, 1741, 1742, 1749, 1750, 1751, 1752, 1753, 1756, 1761, 1782, 1786, 1790, 1795, 1804, 1805, 1806, 1809, 1812, 1821, 1855, 1865, 1869, 1902, 1924, 1935, 1981, 1988, 1991, 1993, 1994, 1997, 1998, 199

In [17]:
lis_out = census_train.iloc[outliers, :]
lis_out

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
5,49.4,58.9,60.1,60.6,59.4,6.6,7.4,7.4,6.1,8.1,...,0.4,0.3,0.5,0.3,0.2,29655.0,32152.0,37785.0,33866.0,29063.0
31,42.0,46.2,51.3,55.6,55.8,8.3,6.4,6.1,5.8,8.5,...,0.6,1.1,0.8,0.7,0.6,20954.0,21804.0,24145.0,26688.0,28826.0
59,50.4,57.9,63.1,65.5,67.3,10.6,11.6,11.4,9.9,10.8,...,1.5,0.7,0.9,0.7,1.4,21663.0,23132.0,24320.0,26150.0,27099.0
67,68.2,67.3,62.4,57.9,57.5,7.7,8.9,8.0,10.5,10.6,...,0.1,0.0,0.0,0.6,0.6,66607.0,69706.0,69250.0,75833.0,72258.0
68,74.1,75.7,77.2,79.0,77.0,12.5,13.1,13.9,14.4,14.4,...,0.8,0.8,0.7,1.3,0.8,85192.0,85517.0,87466.0,87443.0,90708.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121,86.4,89.2,91.0,91.2,91.5,14.0,14.8,13.6,13.2,13.1,...,1.3,1.1,0.7,0.4,0.4,80178.0,79916.0,82659.0,80887.0,87547.0
3124,70.0,72.0,73.5,79.5,82.7,14.2,15.7,16.8,19.2,20.7,...,0.1,1.6,2.2,3.1,3.4,66944.0,64055.0,65132.0,66898.0,67835.0
3136,82.9,81.7,85.6,88.1,89.8,19.2,19.0,16.7,21.7,20.9,...,0.1,0.0,0.0,0.0,0.0,84911.0,78680.0,77403.0,78655.0,82342.0
3139,83.8,88.2,89.5,91.4,90.6,11.9,10.5,11.1,12.6,12.3,...,1.2,1.2,1.4,1.7,0.9,54672.0,58235.0,63403.0,72458.0,75106.0


In [18]:
#Then detected the outliers changing these values by median of column
column = []
for i in census_train.columns:
  column.append(i)
for c in column:
  for o in outliers:
    census_train.loc[o,c] = census_train.loc[:,c].mean()

In [19]:
predict = detector.labels_

In [20]:
census_train.columns

Index(['pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020',
       'pct_bb_2021', 'pct_college_2017', 'pct_college_2018',
       'pct_college_2019', 'pct_college_2020', 'pct_college_2021',
       'pct_foreign_born_2017', 'pct_foreign_born_2018',
       'pct_foreign_born_2019', 'pct_foreign_born_2020',
       'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018',
       'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021',
       'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019',
       'median_hh_inc_2020', 'median_hh_inc_2021'],
      dtype='object')

In [21]:
census_train = census_train.astype('object')

In [22]:
#Reducing the 25 columns for 5 columns
#We have 5 big groups on these groups have 5 columns so I formulated one column per group with the mean of all 5 columns in each group
columns = {'pct_bb_2017': census_train['pct_bb_2017'] ,
           'pct_bb_2018': census_train['pct_bb_2018'] , 
           'pct_bb_2019': census_train['pct_bb_2019'] ,
           'pct_bb_2020': census_train['pct_bb_2020'] ,
           'pct_bb_2021': census_train['pct_bb_2021'] }
name = ['pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021']
mean_pct = []
count = []

for i in range(len(census_train)):
  if i == 0:
    i =1
  elif i == 1:
    i += 1
  elif i> 1:
    i += 1
  count.append(i)

df = pd.DataFrame(columns, count)

columns2 = {'pct_college_2017': census_train['pct_college_2017'] ,
           'pct_college_2018': census_train['pct_college_2018'] , 
           'pct_college_2019': census_train['pct_college_2019'] ,
           'pct_college_2020': census_train['pct_college_2020'] ,
           'pct_college_2021': census_train['pct_college_2021'] }
df2 = pd.DataFrame(columns2, count)

columns3 = {'pct_foreign_born_2017': census_train['pct_foreign_born_2017'] ,
           'pct_foreign_born_2018': census_train['pct_foreign_born_2018'] , 
           'pct_foreign_born_2019': census_train['pct_foreign_born_2019'] ,
           'pct_foreign_born_2020': census_train['pct_foreign_born_2020'] ,
           'pct_foreign_born_2021': census_train['pct_foreign_born_2021'] }
df3 = pd.DataFrame(columns3, count)

columns4 = {'pct_it_workers_2017': census_train['pct_it_workers_2017'] ,
           'pct_it_workers_2018': census_train['pct_it_workers_2018'] , 
           'pct_it_workers_2019': census_train['pct_it_workers_2019'] ,
           'pct_it_workers_2020': census_train['pct_it_workers_2020'] ,
           'pct_it_workers_2021': census_train['pct_it_workers_2021'] }
df4 = pd.DataFrame(columns4, count)


columns5 = {'median_hh_inc_2017': census_train['median_hh_inc_2017'] ,
           'median_hh_inc_2018': census_train['median_hh_inc_2018'] , 
           'median_hh_inc_2019': census_train['median_hh_inc_2019'] ,
           'median_hh_inc_2020': census_train['median_hh_inc_2020'] ,
           'median_hh_inc_2021': census_train['median_hh_inc_2021'] }
df5 = pd.DataFrame(columns5, count)

In [23]:
#Storing the mean columns in a list
df_media_bb = []
for i in df.index:
  df_media_bb.append(df.loc[i].mean())

In [24]:
df_media_college = []
for i in df2.index:
  df_media_college.append(df2.loc[i].mean())

In [25]:
df_media_foreign = []
for i in df3.index:
  df_media_foreign.append(df3.loc[i].mean())

In [26]:
df_media_it = []
for i in df4.index:
  df_media_it.append(df4.loc[i].mean())

In [27]:
df_media_hh = []
for i in df5.index:
  df_media_hh.append(df5.loc[i].mean())

In [28]:
#Puting these mean columns in the dataframe
census_train['pct_bb_mean'] = df_media_bb
census_train['pct_college_mean'] =  df_media_college
census_train['mean_hh_inc'] = df_media_hh
census_train['pct_foreign_born_mean'] = df_media_foreign
census_train['pct_mean_worked'] = df_media_it

In [29]:
census_train.columns

Index(['pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020',
       'pct_bb_2021', 'pct_college_2017', 'pct_college_2018',
       'pct_college_2019', 'pct_college_2020', 'pct_college_2021',
       'pct_foreign_born_2017', 'pct_foreign_born_2018',
       'pct_foreign_born_2019', 'pct_foreign_born_2020',
       'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018',
       'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021',
       'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019',
       'median_hh_inc_2020', 'median_hh_inc_2021', 'pct_bb_mean',
       'pct_college_mean', 'mean_hh_inc', 'pct_foreign_born_mean',
       'pct_mean_worked'],
      dtype='object')

In [30]:
#Deleting all columns just no the mean columns
census_train = census_train.drop(['pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020','pct_bb_2021'], axis=1)
census_train = census_train.drop(['pct_college_2017', 'pct_college_2018','pct_college_2019', 'pct_college_2020', 'pct_college_2021'], axis=1)
census_train = census_train.drop(['pct_foreign_born_2017', 'pct_foreign_born_2018','pct_foreign_born_2019', 'pct_foreign_born_2020','pct_foreign_born_2021'], axis=1)
census_train = census_train.drop(['pct_it_workers_2017', 'pct_it_workers_2018','pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021'], axis=1)
census_train = census_train.drop(['median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019','median_hh_inc_2020', 'median_hh_inc_2021'], axis=1)

In [31]:
census_train.columns

Index(['pct_bb_mean', 'pct_college_mean', 'mean_hh_inc',
       'pct_foreign_born_mean', 'pct_mean_worked'],
      dtype='object')

In [32]:
census_train

,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked
0,81.480000,20.580000,58589.20000,3.440000,1.280000
1,61.460000,7.400000,34298.20000,2.620000,0.700000
2,69.600000,7.500000,48456.80000,1.320000,1.600000
3,73.580000,8.720000,49443.40000,4.460000,1.120000
4,75.424481,14.312511,53603.50956,4.732862,1.346539
...,...,...,...,...,...
3138,75.337758,14.166206,53260.42384,4.580079,1.334793
3139,85.620000,19.500000,80398.20000,4.320000,0.020000
3140,75.335482,14.165252,53258.42372,4.580831,1.334715
3141,87.220000,37.980000,86021.80000,11.260000,1.360000


In [33]:
census_test

array([ 1001.,  1003.,  1005., ..., 56041., 56043., 56045.])

In [34]:
census_test = census_test.reshape(-1,1)

In [35]:
census_train.fillna(0, inplace=True)

In [36]:
census_train

,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked
0,81.480000,20.580000,58589.20000,3.440000,1.280000
1,61.460000,7.400000,34298.20000,2.620000,0.700000
2,69.600000,7.500000,48456.80000,1.320000,1.600000
3,73.580000,8.720000,49443.40000,4.460000,1.120000
4,75.424481,14.312511,53603.50956,4.732862,1.346539
...,...,...,...,...,...
3138,75.337758,14.166206,53260.42384,4.580079,1.334793
3139,85.620000,19.500000,80398.20000,4.320000,0.020000
3140,75.335482,14.165252,53258.42372,4.580831,1.334715
3141,87.220000,37.980000,86021.80000,11.260000,1.360000


# **Train Database**



In [37]:
#2St dataset loading
train = pd.read_csv('/content/drive/MyDrive/train.csv')

In [38]:
train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [39]:
#Inputing the  census dataset at train dataset
train['pct_bb_mean'] = census_train['pct_bb_mean']
train['pct_college_mean'] = census_train['pct_college_mean']
train['mean_hh_inc'] = census_train['mean_hh_inc']
train['pct_foreign_born_mean'] = census_train['pct_foreign_born_mean']
train['pct_mean_worked'] = census_train['pct_mean_worked']

In [40]:
train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,81.480000,20.580000,58589.20000,3.440000,1.280000
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,61.460000,7.400000,34298.20000,2.620000,0.700000
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,69.600000,7.500000,48456.80000,1.320000,1.600000
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,73.580000,8.720000,49443.40000,4.460000,1.120000
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,75.424481,14.312511,53603.50956,4.732862,1.346539
...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaN,NaN,NaN,NaN,NaN
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaN,NaN,NaN,NaN,NaN
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaN,NaN,NaN,NaN,NaN
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaN,NaN,NaN,NaN,NaN


In [41]:
train.drop('row_id', inplace=True, axis=1)

In [42]:
#3St dataset loadinig
rv = pd.read_csv('/content/drive/MyDrive/revealed_test.csv')
rv

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2022-11-01,1001,Autauga County,Alabama,2022-11-01,3.442677,1463
1,1001_2022-12-01,1001,Autauga County,Alabama,2022-12-01,3.470915,1475
2,1003_2022-11-01,1003,Baldwin County,Alabama,2022-11-01,8.257636,14145
3,1003_2022-12-01,1003,Baldwin County,Alabama,2022-12-01,8.250630,14133
4,1005_2022-11-01,1005,Barbour County,Alabama,2022-11-01,1.247223,247
...,...,...,...,...,...,...,...
6265,56041_2022-12-01,56041,Uinta County,Wyoming,2022-12-01,4.050703,588
6266,56043_2022-11-01,56043,Washakie County,Wyoming,2022-11-01,3.143093,190
6267,56043_2022-12-01,56043,Washakie County,Wyoming,2022-12-01,3.060380,185
6268,56045_2022-11-01,56045,Weston County,Wyoming,2022-11-01,1.785395,100


In [43]:
rv.drop('row_id', axis=1, inplace=True)

In [44]:
#Concating these two datasets train and rv
train = pd.concat([train, rv], ignore_index=True)

In [45]:
# Loading the last dataset
test = pd.read_csv('/content/drive/MyDrive/test.csv')
test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [46]:
test.drop('row_id', inplace=True, axis=1)

In [47]:
#Concating the test dataset with train dataset
train = pd.concat([train, test], ignore_index=True)

In [48]:
#Spliting da column first_day_of_month in year, month and day
import datetime as dt
train['first_day_of_month'] = pd.to_datetime(train['first_day_of_month'])
train['year'] = train['first_day_of_month'].dt.year
train['month'] = train['first_day_of_month'].dt.month
train['day'] = train['first_day_of_month'].dt.day

In [49]:
#Using the Label Encoder for transforming the categorical columns in numeric
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
train['county'] = lb.fit_transform(train['county'])
train['state'] = lb.fit_transform(train['state'])

In [50]:
train

,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day
0,1001,82,0,2019-08-01,3.007682,1249.0,81.480000,20.580000,58589.20000,3.440000,1.280000,2019,8,1
1,1001,82,0,2019-09-01,2.884870,1198.0,61.460000,7.400000,34298.20000,2.620000,0.700000,2019,9,1
2,1001,82,0,2019-10-01,3.055843,1269.0,69.600000,7.500000,48456.80000,1.320000,1.600000,2019,10,1
3,1001,82,0,2019-11-01,2.993233,1243.0,73.580000,8.720000,49443.40000,4.460000,1.120000,2019,11,1
4,1001,82,0,2019-12-01,2.993233,1243.0,75.424481,14.312511,53603.50956,4.732862,1.346539,2019,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153610,56037,1871,51,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,6,1
153611,56039,1871,51,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,6,1
153612,56041,1871,51,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,6,1
153613,56043,1871,51,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,6,1


In [51]:
#Changing the NaN values by mean of column
train['microbusiness_density'].fillna(train['microbusiness_density'].mean(), inplace=True)
train['active'].fillna(train['active'].mean(), inplace=True)
train['pct_bb_mean'].fillna(train['pct_bb_mean'].mean(), inplace=True)
train['pct_college_mean'].fillna(train['pct_college_mean'].mean(), inplace=True)
train['mean_hh_inc'].fillna(train['mean_hh_inc'].mean(), inplace=True)
train['pct_foreign_born_mean'].fillna(train['pct_foreign_born_mean'].mean(), inplace=True)
train['pct_mean_worked'].fillna(train['pct_mean_worked'].mean(), inplace=True)

In [52]:
train

,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day
0,1001,82,0,2019-08-01,3.007682,1249.000000,81.480000,20.580000,58589.200000,3.440000,1.280000,2019,8,1
1,1001,82,0,2019-09-01,2.884870,1198.000000,61.460000,7.400000,34298.200000,2.620000,0.700000,2019,9,1
2,1001,82,0,2019-10-01,3.055843,1269.000000,69.600000,7.500000,48456.800000,1.320000,1.600000,2019,10,1
3,1001,82,0,2019-11-01,2.993233,1243.000000,73.580000,8.720000,49443.400000,4.460000,1.120000,2019,11,1
4,1001,82,0,2019-12-01,2.993233,1243.000000,75.424481,14.312511,53603.509560,4.732862,1.346539,2019,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153610,56037,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153611,56039,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153612,56041,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153613,56043,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1


In [53]:
train.columns

Index(['cfips', 'county', 'state', 'first_day_of_month',
       'microbusiness_density', 'active', 'pct_bb_mean', 'pct_college_mean',
       'mean_hh_inc', 'pct_foreign_born_mean', 'pct_mean_worked', 'year',
       'month', 'day'],
      dtype='object')

In [54]:
train

,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day
0,1001,82,0,2019-08-01,3.007682,1249.000000,81.480000,20.580000,58589.200000,3.440000,1.280000,2019,8,1
1,1001,82,0,2019-09-01,2.884870,1198.000000,61.460000,7.400000,34298.200000,2.620000,0.700000,2019,9,1
2,1001,82,0,2019-10-01,3.055843,1269.000000,69.600000,7.500000,48456.800000,1.320000,1.600000,2019,10,1
3,1001,82,0,2019-11-01,2.993233,1243.000000,73.580000,8.720000,49443.400000,4.460000,1.120000,2019,11,1
4,1001,82,0,2019-12-01,2.993233,1243.000000,75.424481,14.312511,53603.509560,4.732862,1.346539,2019,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153610,56037,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153611,56039,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153612,56041,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153613,56043,1871,51,2023-06-01,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1


In [55]:
#Droping the first_day_of_month because we aready have that column splited
train = train.drop('first_day_of_month', axis=1)

In [56]:
train

,cfips,county,state,microbusiness_density,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day
0,1001,82,0,3.007682,1249.000000,81.480000,20.580000,58589.200000,3.440000,1.280000,2019,8,1
1,1001,82,0,2.884870,1198.000000,61.460000,7.400000,34298.200000,2.620000,0.700000,2019,9,1
2,1001,82,0,3.055843,1269.000000,69.600000,7.500000,48456.800000,1.320000,1.600000,2019,10,1
3,1001,82,0,2.993233,1243.000000,73.580000,8.720000,49443.400000,4.460000,1.120000,2019,11,1
4,1001,82,0,2.993233,1243.000000,75.424481,14.312511,53603.509560,4.732862,1.346539,2019,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153610,56037,1871,51,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153611,56039,1871,51,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153612,56041,1871,51,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1
153613,56043,1871,51,3.827817,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,6,1


In [57]:
#Standardizing the values for have a better score
from sklearn.preprocessing import StandardScaler
obg_df = StandardScaler().fit(train)

In [58]:
train_df = obg_df.transform(train)
train_df

array([[-1.93978544, -1.70210471, -1.90020988, ..., -1.74569518,
         0.36706517,  0.        ],
       [-1.93978544, -1.70210471, -1.90020988, ..., -1.74569518,
         0.64809945,  0.        ],
       [-1.93978544, -1.70210471, -1.90020988, ..., -1.74569518,
         0.92913372,  0.        ],
       ...,
       [ 1.69479001,  1.32276573,  1.30093102, ...,  1.60877791,
        -0.19500337,  0.        ],
       [ 1.69492208,  1.32276573,  1.30093102, ...,  1.60877791,
        -0.19500337,  0.        ],
       [ 1.69505415,  1.32276573,  1.30093102, ...,  1.60877791,
        -0.19500337,  0.        ]])

In [59]:
train.columns

Index(['cfips', 'county', 'state', 'microbusiness_density', 'active',
       'pct_bb_mean', 'pct_college_mean', 'mean_hh_inc',
       'pct_foreign_born_mean', 'pct_mean_worked', 'year', 'month', 'day'],
      dtype='object')

In [60]:
#Remaking the dataframe
train_df = pd.DataFrame(train_df)
train_df.columns = ['cfips', 'county', 'state', 'microbusiness_density', 'active',
       'pct_bb_mean', 'pct_college_mean', 'mean_hh_inc',
       'pct_foreign_born_mean', 'pct_mean_worked', 'year', 'month', 'day']

In [61]:
train_df

,cfips,county,state,microbusiness_density,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day
0,-1.939785,-1.702105,-1.900210,-1.772157e-01,-0.172055,5.192038e+00,8.574124e+00,2.862291,-1.525684e+00,-4.824178e-01,-1.745695,0.367065,0.0
1,-1.939785,-1.702105,-1.900210,-2.037530e-01,-0.173739,-1.151077e+01,-9.003201e+00,-10.032018,-2.624490e+00,-5.648146e+00,-1.745695,0.648099,0.0
2,-1.939785,-1.702105,-1.900210,-1.668090e-01,-0.171395,-4.719518e+00,-8.869837e+00,-2.516256,-4.366500e+00,2.367639e+00,-1.745695,0.929134,0.0
3,-1.939785,-1.702105,-1.900210,-1.803378e-01,-0.172253,-1.398980e+00,-7.242801e+00,-1.992543,-1.588758e-01,-1.907446e+00,-1.745695,1.210168,0.0
4,-1.939785,-1.702105,-1.900210,-1.803378e-01,-0.172253,1.398811e-01,2.155743e-01,0.215754,2.067610e-01,1.102088e-01,-1.745695,1.491202,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153610,1.694526,1.322766,1.300931,-9.595930e-17,0.000000,1.185620e-14,-1.184507e-14,0.000000,3.570498e-15,1.977624e-15,1.608778,-0.195003,0.0
153611,1.694658,1.322766,1.300931,-9.595930e-17,0.000000,1.185620e-14,-1.184507e-14,0.000000,3.570498e-15,1.977624e-15,1.608778,-0.195003,0.0
153612,1.694790,1.322766,1.300931,-9.595930e-17,0.000000,1.185620e-14,-1.184507e-14,0.000000,3.570498e-15,1.977624e-15,1.608778,-0.195003,0.0
153613,1.694922,1.322766,1.300931,-9.595930e-17,0.000000,1.185620e-14,-1.184507e-14,0.000000,3.570498e-15,1.977624e-15,1.608778,-0.195003,0.0


In [62]:
#Normalization the values for compete with Standard values
from sklearn.preprocessing import MinMaxScaler
obj_norm = MinMaxScaler().fit(train)
train_norm = obj_norm.transform(train)
train_norm

array([[0.        , 0.04382683, 0.        , ..., 0.        , 0.63636364,
        0.        ],
       [0.        , 0.04382683, 0.        , ..., 0.        , 0.72727273,
        0.        ],
       [0.        , 0.04382683, 0.        , ..., 0.        , 0.81818182,
        0.        ],
       ...,
       [0.99992733, 1.        , 1.        , ..., 1.        , 0.45454545,
        0.        ],
       [0.99996367, 1.        , 1.        , ..., 1.        , 0.45454545,
        0.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 0.45454545,
        0.        ]])

In [63]:
train_norm = pd.DataFrame(train_norm)
train_norm.columns = ['cfips', 'county', 'state', 'microbusiness_density', 'active',
       'pct_bb_mean', 'pct_college_mean', 'mean_hh_inc',
       'pct_foreign_born_mean', 'pct_mean_worked', 'year', 'month', 'day']

In [64]:
train_norm

,cfips,county,state,microbusiness_density,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day
0,0.000000,0.043827,0.0,0.010578,0.001070,0.861493,0.467727,0.437759,0.064203,0.105611,0.0,0.636364,0.0
1,0.000000,0.043827,0.0,0.010146,0.001026,0.649820,0.168182,0.256265,0.048899,0.057756,0.0,0.727273,0.0
2,0.000000,0.043827,0.0,0.010747,0.001087,0.735885,0.170455,0.362053,0.024636,0.132013,0.0,0.818182,0.0
3,0.000000,0.043827,0.0,0.010527,0.001064,0.777966,0.198182,0.369425,0.083240,0.092409,0.0,0.909091,0.0
4,0.000000,0.043827,0.0,0.010527,0.001064,0.797468,0.325284,0.400508,0.088333,0.111101,0.0,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153610,0.999855,1.000000,1.0,0.013462,0.005533,0.795695,0.321611,0.397471,0.085453,0.110080,1.0,0.454545,0.0
153611,0.999891,1.000000,1.0,0.013462,0.005533,0.795695,0.321611,0.397471,0.085453,0.110080,1.0,0.454545,0.0
153612,0.999927,1.000000,1.0,0.013462,0.005533,0.795695,0.321611,0.397471,0.085453,0.110080,1.0,0.454545,0.0
153613,0.999964,1.000000,1.0,0.013462,0.005533,0.795695,0.321611,0.397471,0.085453,0.110080,1.0,0.454545,0.0


In [65]:
#Spiting the dataset in train and test, the train part we have 2 variables one with standard values and other with normalization values
var_predict = train_df.drop('microbusiness_density', axis=1)
var_target = train['microbusiness_density']
var_predict_norm = train_norm.drop('microbusiness_density', axis=1)

In [66]:
x_train, x_test, y_train, y_test = train_test_split(var_predict, var_target, test_size=0.3, random_state=1)

In [67]:
#See the performance of XGBRegressor with standard values
xgb = xgboost.XGBRegressor()
xgb = xgb.fit(x_train, y_train)

In [68]:
#Score: 96.10
ac = xgb.score(x_test, y_test)
print('Accurancy :' + str(ac))

Accurancy :0.9610957638487205


In [69]:
#See the performance of XGBRegressor with standard values
#Score: 96.10
x_train, x_test, y_train, y_test = train_test_split(var_predict_norm, var_target, test_size=0.3, random_state=1)
xgb = xgboost.XGBRegressor()
xgb = xgb.fit(x_train, y_train)
ac = xgb.score(x_test, y_test)
print('Accurancy :' + str(ac))

Accurancy :0.9610886927386596


In [70]:
#See the performance wih RandomFlorestRegressor with standard values
#Score: 99.38
x_train, x_test, y_train, y_test = train_test_split(var_predict, var_target, test_size=0.3, random_state=1)
rf = RandomForestRegressor()
rf = rf.fit(x_train, y_train)
ac = rf.score(x_test, y_test)
print('Accurancy :' + str(ac))

Accurancy :0.9940317802714933


In [71]:
#extracting the predictions
predict = rf.predict(x_test)

In [72]:
predict

array([3.4787716 , 0.87832221, 3.8278169 , ..., 5.39228596, 0.98279908,
       4.36021603])

In [73]:
len(predict)

46085

In [74]:
#Go back with original values
var_train = train.drop('microbusiness_density', axis=1)
x_train, x_test, y_train, y_test = train_test_split(var_train, var_target, test_size=0.3, random_state=1)

In [75]:
x_test

,cfips,county,state,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day
114294,51700,1188,46,4845.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2021,8,1
83034,40003,21,36,39.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2019,11,1
149703,45091,1871,51,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,5,1
110728,51047,432,46,3403.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2020,3,1
73900,37015,146,33,230.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2022,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...
83118,40007,122,36,84.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2020,5,1
128265,54031,727,48,253.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2022,11,1
106524,48425,1554,43,367.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2020,11,1
116688,54015,350,48,69.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2019,8,1


In [76]:
#Puting the values of predict
x_test['microbusiness_density'] = predict

In [77]:
#The column "predict" is reverent the microbusiness_density that is the column target
x_test

,cfips,county,state,active,pct_bb_mean,pct_college_mean,mean_hh_inc,pct_foreign_born_mean,pct_mean_worked,year,month,day,microbusiness_density
114294,51700,1188,46,4845.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2021,8,1,3.478772
83034,40003,21,36,39.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2019,11,1,0.878322
149703,45091,1871,51,6461.169277,75.256819,14.150867,53197.060555,4.578563,1.334165,2023,5,1,3.827817
110728,51047,432,46,3403.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2020,3,1,9.018180
73900,37015,146,33,230.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2022,6,1,1.442895
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83118,40007,122,36,84.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2020,5,1,2.060255
128265,54031,727,48,253.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2022,11,1,2.309525
106524,48425,1554,43,367.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2020,11,1,5.392286
116688,54015,350,48,69.000000,75.256819,14.150867,53197.060555,4.578563,1.334165,2019,8,1,0.982799


In [80]:
#The MAE is 0.04
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predict)

print("MAE:", mae)

MAE: 0.047736741176706565


Thank you for see my code